In [0]:
import time
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

In [0]:
URL_Train= "https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv"
URL_Test="https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/test.tsv"

In [0]:
train=pd.read_csv(URL_Train,sep='\t')
test= pd.read_csv(URL_Test, sep='\t')

In [0]:
fullSent = train.loc[train.groupby('SentenceId')['PhraseId'].idxmin()]
fullSent.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
63,64,2,"This quiet , introspective and entertaining in...",4
81,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1
116,117,4,A positively thrilling combination of ethnogra...,3
156,157,5,Aggressive self-glorification and a manipulati...,1


In [0]:
BoW_vectorizer = CountVectorizer(strip_accents='unicode',
                                 ngram_range=(1,3),
                                 analyzer='word',
                                 min_df=5,
                                 max_df=0.5)

In [0]:
BoW_vectorizer.fit(list(fullSent['Phrase']))

fullSent.head()


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
63,64,2,"This quiet , introspective and entertaining in...",4
81,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1
116,117,4,A positively thrilling combination of ethnogra...,3
156,157,5,Aggressive self-glorification and a manipulati...,1


In [0]:

phrase = fullSent['Phrase']
sentiment = fullSent['Sentiment']

phrase.shape, sentiment.shape


((8529,), (8529,))

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(phrase,
                                                    sentiment,
                                                    test_size=0.2,
                                                    random_state=4)



X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((6823,), (6823,), (1706,), (1706,))

In [0]:
from keras.utils import to_categorical
y_train = to_categorical(Y_train)
y_train.shape

y_test = to_categorical(Y_test)
y_test.shape

(1706, 5)

In [0]:
  train_bow= BoW_vectorizer.transform(X_train)
  train_bow= np.array(train_bow.toarray())
  train_bow = train_bow.reshape(train_bow.shape[0],train_bow.shape[1],1)
  train_bow.shape

(6823, 6855, 1)

In [0]:
test_bow= BoW_vectorizer.transform(X_test)
test_bow_reshape= np.array(test_bow.toarray())
test_bow_reshape= test_bow_reshape.reshape(test_bow_reshape.shape[0],test_bow_reshape.shape[1],1)
test_bow_reshape.shape

(1706, 6855, 1)

In [0]:
bow_feature_vec = pd.DataFrame(test_bow.toarray(), columns= BoW_vectorizer.get_feature_names())
bow_feature_vec.head(25)

,10,10 minutes,100,101,11,12,13,15,19,20,20 years,2002,20th,21st,30,50,51,60s,70s,80,90,90 minute,90 minutes,90 minutes of,abandon,ability,ability to,able,able to,about,about an,about as,about growing,about how,about it,about its,about one,about schmidt,about something,about the,...,you might,you might want,you re,you re in,you re looking,you re not,you see,you should,you think,you think you,you thought,you to,you ve,you ve got,you ve seen,you want,you want to,you were,you will,you wish,you with,you wo,you would,young,young men,young woman,younger,your,your face,your head,your heart,your own,your seat,your time,yourself,youth,yu,zero,zhang,zone
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
from keras import backend as K
def recall_m(y_true,y_pred):
  true_positives = K.sum(K.round(K.clip(y_true*y_pred,0,1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0,1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

def precision_m(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true*y_pred, 0,1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision

def f1_m(y_true, y_pred):
  precision= precision_m(y_true, y_pred)
  recall= recall_m(y_true, y_pred)
  return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Flatten
from keras.layers import Activation, Conv1D, GlobalMaxPooling1D
from keras import optimizers
from keras import layers

In [0]:
def cnn_model(fea_matrix, n_class, mode,compiler):
  model= Sequential()
  model.add(Conv1D(filters=64, kernel_size=2, activation= 'relu', input_shape= (fea_matrix.shape[1],fea_matrix.shape[2])))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Conv1D(filters=64, kernel_size=2, activation= 'relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Conv1D(filters=64, kernel_size=2, activation= 'relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Activation('relu'))
  model.add(Dense(n_class))
  if n_class == 1 and mode =="cla":
    model.add(Activation('sigmoid'))
    model.compile(optimizer= compiler, loss='binary_crossentropy', metrics=['acc', f1_m, precision_m, recall_m])
  else:
    model.add(Activation('softmax'))
    model.compile(optimizer= compiler, loss='categorical_crossentropy', metrics=['acc', f1_m, precision_m, recall_m])
    
  return model

In [0]:
lr=1e-3
batch_size= 64
num_epochs= 100
decay=1e-4
mode= "reg"
n_class = 5
adm = optimizers.Adam(lr=lr, decay=decay)
sgd= optimizers.SGD(lr=lr, nesterov= True, momentum=0.7, decay=decay)
Nadam= optimizers.Nadam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon= 1e-08)
model=cnn_model(train_bow, n_class, mode, Nadam)

In [0]:
Y_train = to_categorical(y_train)
model.fit(train_bow, y_train, batch_size=batch_size, epochs=num_epochs, verbose=1, validation_split=0.2)

Train on 5458 samples, validate on 1365 samples
Epoch 1/10
5458/5458 [==============================] - 7s 1ms/step - loss: 1.5060 - acc: 0.3340 - f1_m: 0.0411 - precision_m: 0.3269 - recall_m: 0.0225 - val_loss: 1.5183 - val_acc: 0.3267 - val_f1_m: 0.0775 - val_precision_m: 0.4295 - val_recall_m: 0.0432
Epoch 2/10
5458/5458 [==============================] - 6s 1ms/step - loss: 1.2549 - acc: 0.4771 - f1_m: 0.3232 - precision_m: 0.6221 - recall_m: 0.2208 - val_loss: 1.5126 - val_acc: 0.3421 - val_f1_m: 0.1618 - val_precision_m: 0.4128 - val_recall_m: 0.1018
Epoch 3/10
5458/5458 [==============================] - 6s 1ms/step - loss: 1.0157 - acc: 0.5885 - f1_m: 0.5235 - precision_m: 0.7183 - recall_m: 0.4166 - val_loss: 1.6734 - val_acc: 0.3443 - val_f1_m: 0.2596 - val_precision_m: 0.3745 - val_recall_m: 0.2000
Epoch 4/10
5458/5458 [==============================] - 6s 1ms/step - loss: 0.7330 - acc: 0.7189 - f1_m: 0.6975 - precision_m: 0.8016 - recall_m: 0.6189 - val_loss: 1.9682 - val_

In [0]:
def get_metrics (accuracy,f1_score,precision,recall):
  print('CNN model performance')
  print('Accuracy:', np.round(accuracy,4))
  print('Precision: ', np.round(precision,4))
  print("recall:", np.round(recall,4 ))
  print("f1score:", np.round(f1_score,4 ))
loss, accuracy, f1_score, precision, recall = model.evaluate(test_bow_reshape, y_test, verbose=False)

In [0]:
get_metrics(accuracy,f1_score, precision, recall)

CNN model performance
Accuracy: 0.357
Precision:  0.3598
recall: 0.3499
f1score: 0.3548
